In [1]:
import os
import pickle
import numpy as np
import torch
import sys
sys.path.append('../inverse_rendering/')
machine_path = os.getcwd()
user_root_dir = '/'.join(machine_path.split('/')[:-2])
sys.path.insert(0,'%s/redner/'%user_root_dir)
import redner
import pyredner
print(pyredner.__file__)
pyredner.render_pytorch.print_timing = False
from general_imports import *

/net/storage001.ib.cluster/om2/user/smadan/redner/pyredner/__init__.py
/net/storage001.ib.cluster/om2/user/smadan/redner/pyredner/__init__.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader/multi_attribute_loader.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader/loader.py
/net/storage001.ib.cluster/om2/user/smadan/training_scaffold_own/res/loader


In [2]:
with open('../training_models/shapenet_class_num_to_class_name.p','rb') as F:
    shapenet_class_num_to_class_name = pickle.load(F)
with open('../training_models/shapenet_id_to_class_num.p','rb') as F:
    shapenet_id_to_class_num = pickle.load(F)

In [3]:
shapenet_class_num_to_class_name

{0: 'airplane,aeroplane,plane',
 1: 'bed',
 2: 'car,auto,automobile,machine,motorcar',
 3: 'chair',
 4: 'guitar',
 5: 'knife',
 6: 'motorcycle,bike',
 7: 'piano,pianoforte,forte-piano',
 8: 'pistol,handgun,side arm,shooting iron',
 9: 'sofa,couch,lounge',
 10: 'table'}

In [7]:
model_path = '../training_models/saved_models/resnet18_v7_normalized_final.'
loaded_model = torch.load(model_path)
softmax_layer = nn.Softmax()

In [8]:
def clone_scene_params(scene_params):
    original_scene_params = [0,0,0,0]
    original_scene_params[0] = scene_params[0]

    original_scene_params[1] = {}
    for key in scene_params[1].keys():
        original_scene_params[1][key] = scene_params[1][key].clone()

    original_scene_params[2] = {}
    for key in scene_params[2].keys():
        original_scene_params[2][key] = [i.clone() for i in scene_params[2][key]]

    original_scene_params[3] = scene_params[3]
    return original_scene_params

def start_up(scene_params, optimized_params):
    model_file, camera_params, light_params, material_settings = scene_params
    variables = []
    var_names_list = []
    for param in optimized_params:
        if param in camera_params.keys():
            optimize_flags(camera_params[param])
            if type(camera_params[param]) == list:
                variables.extend(camera_params[param])
            else:
                variables.append(camera_params[param])
        elif param in light_params.keys():
            optimize_flags(light_params[param])
            if type(light_params[param]) == list:
                variables.extend(light_params[param])
            else:
                variables.append(light_params[param])

    scene_params = [model_file, camera_params, light_params, material_settings]
    scene = setup_scene(scene_params)

    return scene, variables

def load_geometry(model_file, geometry, mat):
    if geometry:
        obj_model_all = model_file
        obj_model = [i for i in obj_model_all if len(i.vertices)>0]
    else:
        obj_model_all = pyredner.load_obj(model_file, return_objects=True)
        obj_model = [i for i in obj_model_all if len(i.vertices)>0]

    for part in obj_model:
        part.material = mat

    return obj_model

def render_input(scene):
    img = pyredner.render_pathtracing(scene = scene, num_samples = 512, seed = 1)
    img = torch.clamp(img, min = 0.00000001)
    img = torch.pow(img, 1.0/2.2)
    img = img*255/torch.max(img)
    inputs = img.permute(2,0,1).unsqueeze(0)
    return inputs, img

def setup_scene(scene_params):

    model_file, camera_params, light_params, material_settings = scene_params
    obj_model = load_geometry(model_file, False, material_settings['diffuse_material'])

    #### Camera Setup ####
    scene_cam = pyredner.automatic_camera_placement(obj_model, resolution = (224, 224),
                                                   fov = torch.tensor([camera_params['fov']]),
                                                   up = camera_params['cam_up'],
                                                   look_at = camera_params['cam_look_at'])
    scene_cam.position = camera_params['camera_position']

    #### Lights Setup ####
    scene_lights = []
    num_lights = len(light_params['all_light_positions'])
    for i in range(num_lights):
        scene_light = pyredner.generate_quad_light(position = light_params['all_light_positions'][i],
                                     look_at = light_params['all_light_look_ats'][i],
                                     size = light_params['all_light_sizes'][i],
                                     intensity = light_params['all_light_intensities'][i],
                                     directly_visible = False)
        scene_lights.append(scene_light)

    all_objects = obj_model + scene_lights
    scene = pyredner.Scene(objects = all_objects, camera = scene_cam)

    return scene

def show_inputs(inp, save_path = False, title='No Title'):
    plt.imshow(inp[0].cpu().permute(1,2,0).int())
    plt.title(title)
    plt.axis('off')
    if save_path != False:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

In [9]:
# file = os.listdir(folder)[0]

In [12]:
varying_camera = False
varying_light = True

In [13]:
def plot_exp(exp_path, save_folder):
    with open(exp_path, 'rb') as F:
        exp_info = pickle.load(F)

    starting_scene_params = exp_info['scene_params']
    

In [14]:
def evaluate_exp(exp_path, render = False, show = False):
    with open(exp_path, 'rb') as F:
        exp_info = pickle.load(F)
    
    starting_scene_params = exp_info['scene_params']
    all_predictions = exp_info['es'].predictions
    all_prediction_settings = exp_info['es'].prediction_settings
    correct_prediction = exp_info['es'].correct_prediction

    ids = np.where(np.array(all_predictions) != correct_prediction)
    
    if len(ids[0]) != 0:
        found_adv = True
        adv_predictions = np.array(all_predictions)[ids]
        adv_prediction_settings = np.array(all_prediction_settings)[ids]

        adv_id = random.choice(range(len(ids)))

        starting_scene_params = exp_info['scene_params']

        adv_scene_params = clone_scene_params(starting_scene_params)
        if varying_camera:
            adv_scene_params[1]['camera_position'] = torch.tensor(adv_prediction_settings[adv_id][:3]).float()
            adv_scene_params[1]['cam_look_at'] = torch.tensor(adv_prediction_settings[adv_id][3:6]).float()
            adv_scene_params[1]['cam_up'] = torch.tensor(adv_prediction_settings[adv_id][6:9]).float()
            adv_scene_params[1]['fov'] = torch.tensor(adv_prediction_settings[adv_id][9]*10).float()
        elif varying_light:
            num_lights = int(adv_prediction_settings[adv_id].shape[0]/11)
            adv_scene_params[2]['all_light_positions'] = torch.tensor(adv_prediction_settings[adv_id][:num_lights*3]).float().view(num_lights,-1)
            adv_scene_params[2]['all_light_look_ats'] = torch.tensor(adv_prediction_settings[adv_id][num_lights*3:num_lights*6]).float().view(num_lights,-1)
            adv_scene_params[2]['all_light_intensities'] = torch.tensor(adv_prediction_settings[adv_id][num_lights*6:num_lights*9]).float().view(num_lights,-1)
            adv_scene_params[2]['all_light_sizes'] = torch.tensor(adv_prediction_settings[adv_id][num_lights*9:]).float().view(num_lights,-1)
        params = [starting_scene_params, adv_scene_params]
    else:
        found_adv = False
        params = [starting_scene_params]
    
    inps = []
    predictions = []
    imgs = []
    if render:
        print('Rendering')
        for i in range(len(params)):
            param = params[i]
            scene, variables = start_up(param, [])

            inputs, img = render_input(scene)
            imgs.append(img)
            
            inputs = inputs.cuda()
            rendered_inputs = inputs.clone()
            inputs = inputs/255.0
            im_means = torch.mean(inputs).unsqueeze(0).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            im_stds = torch.std(inputs).unsqueeze(0).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            inputs_ = (inputs - im_means)/im_stds
            outputs = loaded_model(inputs_)
            probability = outputs[0][correct_prediction].item()
            prediction = torch.argmax(outputs[0]).item()
            predictions.append(prediction)
            inps.append(rendered_inputs)    
            
        if len(inps) == 1:
            inps.append(inps[0])
            predictions.append(predictions[0])
    
    if show:
        print('Showing')
        fig,ax = plt.subplots(nrows=1,ncols=2)
        ax[0].imshow(inps[0][0].cpu().permute(1,2,0).int())
        ax[0].axis('off')
        ax[0].set_title('Starting: %s'%(shapenet_class_num_to_class_name[predictions[0]].split(',')[0]))
        ax[1].imshow(inps[1][0].cpu().permute(1,2,0).int())
        ax[1].axis('off')        
        ax[1].set_title('Adversarial: %s'%(shapenet_class_num_to_class_name[predictions[1]].split(',')[0]))
        plt.show()
        
    return found_adv, inps, predictions, imgs


In [15]:
# os.listdir('/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_cam_unseen')

In [16]:
# folder = '/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_cam_unseen/resnet18_v7/'
# folder = '/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_cam_unseen/resnet18_v7_40/'
# folder = '/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_cam_unseen/resnet18_v7_subsampled/'
# folder = '/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_cam_unseen/resnet18_v7_truly_shift_invariant/'
folder = '/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_light/neurips_results/resnet18_antialiased_v7_40_normalized_final/'

In [17]:
save_folder = "%s/saved_images/"%folder
if not os.path.isdir(save_folder):
    os.mkdir(save_folder)

In [18]:
rendered_save_folder = "%s/saved_renders/"%folder

In [19]:
if not os.path.isdir(rendered_save_folder):
    os.mkdir(rendered_save_folder)

In [20]:
rendered_save_folder

'/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_light/neurips_results/resnet18_antialiased_v7_40_normalized_final//saved_renders/'

In [21]:
rendered_save_folder

'/om5/user/smadan/differentiable_graphics_ml/other_optimization_methods/cma_adversarial_light/neurips_results/resnet18_antialiased_v7_40_normalized_final//saved_renders/'

In [22]:
len(os.listdir(folder))

48

In [24]:
# chosen_list = ["JPcevyd9BJiO9m8r", "nDS9g1aQFy0cuAnu", "qwJOCIXZJ2HWjq84", "zf8s08TeeTdJLbuF"]

In [25]:
# exp = "zf8s08TeeTdJLbuF.p"
# exp_path = "%s/%s"%(folder, exp)

In [26]:
# with open(exp_path, 'rb') as F:
#     exp_info = pickle.load(F)

In [148]:
starting_scene_params = exp_info['scene_params']
all_predictions = exp_info['es'].predictions
all_prediction_settings = exp_info['es'].prediction_settings
correct_prediction = exp_info['es'].correct_prediction

ids = np.where(np.array(all_predictions) != correct_prediction)

if len(ids[0]) != 0:
    found_adv = True
    adv_predictions = np.array(all_predictions)[ids]
    adv_prediction_settings = np.array(all_prediction_settings)[ids]

    adv_id = random.choice(range(len(ids)))

    starting_scene_params = exp_info['scene_params']

    adv_scene_params = clone_scene_params(starting_scene_params)
    if varying_camera:
        adv_scene_params[1]['camera_position'] = torch.tensor(adv_prediction_settings[adv_id][:3]).float()
        adv_scene_params[1]['cam_look_at'] = torch.tensor(adv_prediction_settings[adv_id][3:6]).float()
        adv_scene_params[1]['cam_up'] = torch.tensor(adv_prediction_settings[adv_id][6:9]).float()
        adv_scene_params[1]['fov'] = torch.tensor(adv_prediction_settings[adv_id][9]*10).float()

In [151]:
starting_scene_params[1].keys()

dict_keys(['camera_position', 'cam_look_at', 'fov', 'cam_up'])

In [149]:
for key in starting_scene_params[1].keys():
    try:
        A = list(np.array(starting_scene_params[1][key]))
        print('%.02f, %.02f, %.02f'%(A[0], A[1], A[2]))
    except:
        print("%.02f"%np.array(starting_scene_params[1][key]))

-1.32, -0.65, -0.68
0.26, 0.37, 0.04
80.09
-0.34, -0.83, -0.52


In [150]:
for key in adv_scene_params[1].keys():
    try:
        A = list(np.array(adv_scene_params[1][key]))
        print('%.02f, %.02f, %.02f'%(A[0], A[1], A[2]))
    except:
        print("%.02f"%np.array(adv_scene_params[1][key]))

-1.25, -0.72, -0.69
0.30, 0.33, 0.06
80.18
-3.36, -8.30, -5.20


In [79]:
# os.path.isfile(exp_p)

In [76]:
# success_count = 0
# total = 0
# for exp in tqdm(os.listdir(folder)):
#     if '.p' in exp:
#         exp_p = "%s/%s"%(folder, exp)        
#         found_adv, rendered_inputs, predictions, images = evaluate_exp(exp_p, render = True, show = False)
        
# #         starting_image_path = "%s/%s_starting_%s.png"%(save_folder, exp_p.split('/')[-1].split('.p')[0], predictions[0])
# #         show_inputs(rendered_inputs[0], save_path = starting_image_path)
        
#         starting_render_path = "%s/%s_starting_%s.png"%(rendered_save_folder, exp_p.split('/')[-1].split('.p')[0], predictions[0])
#         start_image = Image.fromarray(images[0].cpu().numpy().astype('uint8'))
#         start_image.save(starting_render_path)

#         if found_adv == True:
# #             adv_image_path = "%s/%s_adversarial_%s.png"%(save_folder, exp_p.split('/')[-1].split('.p')[0], predictions[1])
# #             show_inputs(rendered_inputs[1], save_path = adv_image_path)

#             adv_render_path = "%s/%s_adversarial_%s.png"%(rendered_save_folder, exp_p.split('/')[-1].split('.p')[0], predictions[1])
#             adv_image = Image.fromarray(images[1].cpu().numpy().astype('uint8'))
#             adv_image.save(adv_render_path)
#             success_count += 1
#         if total == 105:
#             break
#         total += 1

In [159]:
other_count

0

In [160]:
success_count

79

In [161]:
1-success_count/(total+3)

0.28828828828828834